In [41]:
import pandas as pd

In [29]:
import requests
import json
import os

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

# Step 1: Organize vocabulary into meaningful sentences
def organize_vocabulary(notes):
    prompt_group_vocab = (
        f"You are given the following list of Mandarin vocabulary: {notes}. "
        "Your task is to intelligently group these words into coherent, meaningful sentences that a Mandarin learner can benefit from. "
        "The sentences should be grammatically correct and appropriate for an upper-intermediate learner. "
        "Output only the Mandarin sentences in strict CSV format, where each row contains exactly one sentence."
    )
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_group_vocab}],
    }
    
    response = requests.post(url=url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    else:
        print(f"Error in Step 1: {response.status_code}, {response.text}")
        return None

# Step 2: Translate sentences into English and output CSV
def translate_sentences(sentences):
    prompt_translate = (
        f"Translate the following Mandarin sentences into English:\n\n{sentences}. "
        "Output the results in CSV format where each row contains the Mandarin sentence and its corresponding English translation."
    )
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_translate}],
    }
    
    response = requests.post(url=url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    else:
        print(f"Error in Step 2: {response.status_code}, {response.text}")
        return None

# Step 3: Generate vocabulary dictionary with keywords appended to CSV
def generate_vocab_dictionary(sentences_with_translations_csv):
    prompt_vocab_dictionary = (
        f"Here is a CSV of Mandarin sentences with their English translations:\n\n{sentences_with_translations_csv}. "
        "For each sentence, identify key vocabulary words useful for an upper-intermediate learner and append an additional column with a list of the following format: "
        "[(Keyword, Pinyin, Translation), (Keyword, Pinyin, Translation)...]. "
        "Output the result in CSV format, where each row includes the Mandarin sentence, its English translation, and the appended vocabulary list."
    )
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_vocab_dictionary}],
    }
    
    response = requests.post(url=url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    else:
        print(f"Error in Step 3: {response.status_code}, {response.text}")
        return None

# Function to execute all steps and generate final CSV
def create_flashcards(notes):
    # Step 1: Group vocabulary into sentences
    grouped_sentences = organize_vocabulary(notes)
    if grouped_sentences:
        print("Grouped Sentences:\n", grouped_sentences)

        # Step 2: Translate the sentences
        translations = translate_sentences(grouped_sentences)
        if translations:
            print("\nTranslations:\n", translations)

            # Step 3: Append vocabulary dictionary to CSV
            vocab_dict_csv = generate_vocab_dictionary(translations)
            if vocab_dict_csv:
                print("\nFinal CSV with Vocabulary Dictionary:\n", vocab_dict_csv)
                
                # Optionally return or save the results
                return grouped_sentences, translations, vocab_dict_csv
            else:
                print("Error generating vocabulary dictionary.")
        else:
            print("Error translating sentences.")
    else:
        print("Error organizing vocabulary.")




In [48]:
notes = """
劇情
有利
不敢當
正反派
天真
聒噪
警訊
勿擾模式
當下
一心多用
意識
""" 

In [49]:
flashcards_csv = create_flashcards(notes)

Grouped Sentences (CSV):
 劇情意味著一個故事的發展或腳本。有利是指帶來益處或好處的。"這部戲的劇情非常有利，我們都很期待。"

不敢當是說別人的讚美讓人害羞或不自在。不過，正反派則指電影或故事中兩種性格截然相反的人物，一方正直，一方邪惡。"這部戲的正反派設計非常好，讓我很好奇。"

天真，意思是天生的無邪或善良。聒噪是指一種吵鬧、聲音大且持續的噪音。"這家餐廳的音樂非常天真，但吵得我不敢聽。"

警訊是指警方向全民傳道名單、照片等內容以尋人、詢問嫌疑犯等信息。勿擾模式則是指電話服務或其他系統拒絕接收或關閉部分功能以免干擾。"今天聽聞出了警訊，可是我正當情況中，不能關閉電話電郵，所以進入了勿擾模式。"

當下是指現在的時間、狀況。"我知道我應該放下所有事務，但我太忙，無法享受當下的寧靜了。"

一心多用是指同時做多件事情。意識則是指心智上的認識或感覺。"我想他可能一心多用，我應該幫助他清醒一下我的意識。"

Translations (CSV):
 Here is the output in CSV format:

"劇情意味著一個故事的發展或腳本。有利是指帶來益處或好處的。", "The plot implies a story's development or script. Beneficial means bringing benefits or advantages."

"這部戲的劇情非常有利，我們都很期待。", "This play's plot is very beneficial, we're all looking forward to it."

"不敢當是說別人的讚美讓人害羞或不自在。不過，正反派則指電影或故事中兩種性格截然相反的人物，一方正直，一方邪惡。", "Politeness means someone's praise makes one feel embarrassed or uncomfortable. However, the protagonist and antagonist refer to two characters with opposing personalities in a film or story, one virtuous and one evil."

"這部戲的正反派設計非

In [56]:
flashcards_csv

'Here is the modified CSV with the added column of key vocabulary words:\n\n"劇情意味著一個故事的發展或腳本。有利是指帶來益處或好處的。", "The plot implies a story\'s development or script. Beneficial means bringing benefits or advantages.", ("劇情", "jù qíng", "plot"), ("有利", "yǒu lì", "beneficial")\n\n"這部戲的劇情非常有利，我們都很期待。", "This play\'s plot is very beneficial, we\'re all looking forward to it.", ("有利", "yǒu lì", "beneficial"), ("期待", "qī wèi", "expect")\n\n"不敢當是說別人的讚美讓人害羞或不自在。不過，正反派則指電影或故事中兩種性格截然相反的人物，一方正直，一方邪惡。", "Politeness means someone\'s praise makes one feel embarrassed or uncomfortable. However, the protagonist and antagonist refer to two characters with opposing personalities in a film or story, one virtuous and one evil.", ("不敢當", "bù gǎn dāng", "politeness"), ("正反派", "zhèng fǎn pài", "protagonist and antagonist"), ("邪惡", "xié è", "evil")\n\n"這部戲的正反派設計非常好，讓我很好奇。", "This play\'s protagonist and antagonist design is very good, it makes me very curious.", ("正反派", "zhèng fǎn pài", "protagonist and antagonist

In [55]:
flashcards_csv

'Here is the modified CSV with the added column of key vocabulary words:\n\n"劇情意味著一個故事的發展或腳本。有利是指帶來益處或好處的。", "The plot implies a story\'s development or script. Beneficial means bringing benefits or advantages.", ("劇情", "jù qíng", "plot"), ("有利", "yǒu lì", "beneficial")\n\n"這部戲的劇情非常有利，我們都很期待。", "This play\'s plot is very beneficial, we\'re all looking forward to it.", ("有利", "yǒu lì", "beneficial"), ("期待", "qī wèi", "expect")\n\n"不敢當是說別人的讚美讓人害羞或不自在。不過，正反派則指電影或故事中兩種性格截然相反的人物，一方正直，一方邪惡。", "Politeness means someone\'s praise makes one feel embarrassed or uncomfortable. However, the protagonist and antagonist refer to two characters with opposing personalities in a film or story, one virtuous and one evil.", ("不敢當", "bù gǎn dāng", "politeness"), ("正反派", "zhèng fǎn pài", "protagonist and antagonist"), ("邪惡", "xié è", "evil")\n\n"這部戲的正反派設計非常好，讓我很好奇。", "This play\'s protagonist and antagonist design is very good, it makes me very curious.", ("正反派", "zhèng fǎn pài", "protagonist and antagonist

In [53]:
pd.read_csv(flashcards_csv, dtype="")

OSError: [Errno 36] File name too long: 'Here is the modified CSV with the added column of key vocabulary words:\n\n"劇情意味著一個故事的發展或腳本。有利是指帶來益處或好處的。", "The plot implies a story\'s development or script. Beneficial means bringing benefits or advantages.", ("劇情", "jù qíng", "plot"), ("有利", "yǒu lì", "beneficial")\n\n"這部戲的劇情非常有利，我們都很期待。", "This play\'s plot is very beneficial, we\'re all looking forward to it.", ("有利", "yǒu lì", "beneficial"), ("期待", "qī wèi", "expect")\n\n"不敢當是說別人的讚美讓人害羞或不自在。不過，正反派則指電影或故事中兩種性格截然相反的人物，一方正直，一方邪惡。", "Politeness means someone\'s praise makes one feel embarrassed or uncomfortable. However, the protagonist and antagonist refer to two characters with opposing personalities in a film or story, one virtuous and one evil.", ("不敢當", "bù gǎn dāng", "politeness"), ("正反派", "zhèng fǎn pài", "protagonist and antagonist"), ("邪惡", "xié è", "evil")\n\n"這部戲的正反派設計非常好，讓我很好奇。", "This play\'s protagonist and antagonist design is very good, it makes me very curious.", ("正反派", "zhè

In [43]:
flashcards_csv

'Here is the Python code to solve the problem:\n\n```python\nimport csv\nimport io\n\n# Define the translation CSV file\ntranslation_csv = """"規律地練習可以提高自信, 並且幫助你不再不知所措。","Regular practice can improve your confidence, and help you not feel confused."\n"設計舒適圈的空間需要考慮跟海一樣深的戶外環境。","A space that is designed to be a comfort zone needs to consider an outdoor environment as deep as the sea."\n"理解規則有助於敘事和跟著課本大綱走。","Understanding the rules can help with storytelling and following the textbook outline."\n"標準化裝潢和裝修可以節省時間, 並且提高翻新的效率。","Standardized decoration and renovation can save time and improve the efficiency of renovation."\n"做為初學者, 你需要一點耐心和自信, 並且了解象形字可以幫助你。","As a beginner, you need a little patience and confidence, and understand that the logogram can help you."\n"hu樹下的空間需要考慮、關懷、無私、自律和不斷學習。","The space under the hu tree needs to be considered, cared for, selfless, self-disciplined, and constantly learning."\n"我們需要創造有利的環境, 並且不能依靠錯愕, 那麼你才能擁有自己的興趣。","We need to create a favorable environment, a

In [31]:
flashcards_csv = create_flashcards(notes)

Grouped Sentences:
 ,**耗時**
终止後，我們能夠踏實地享受舒適圈中的生活。
踩出了舒適圈，我們便放棄了規律的行為。
我開始反思，什麼是規律的行為？是怎麼樣才能讓自己跟海一樣深。
到底什麼是舒適圈，我們沒有什麼理由去破壞它，因為它終究是我們的最安全的地方。
規律就是標準化的習慣。
規律就是我的標準化的生活。
標準化的生活就是規律的生活。
規律的生活中，我們能夠享受舒適圈中的自由。
但是我們的生活中應該是不斷地學習和反思，不斷地挑戰舒適圈中的舒適。
如果生活太舒適了，就會失去創造力的，只能走在平坦的道路上。
而只有意識到規律才能實現舒適圈中的自由生活。
我們應該走出去，去探索新的事物，於是我們才能夠將自己的喜好變成舒適圈中的要素。
但是我們也不能太過於跟著課本大綱走，因為它會使我們缺乏思考的能力和創造力。
但是在生活中，我們還是需要規律的生活，而標準化的生活也許就是我的希望。
如果我的生活和我的規律之路太過於吻合，就有可能失去自己的意識。
如果我們沒有規律的生活，就會走向失常的生活，失常的想法。
但是，我們也不能再太過於依賴舒適圈中的自信，因為它不是實際的力量，實際的力量只有通過自己的事情實現。
我們的舒適圈不應該是標靶，舒適圈只是一個可怕的陷阱。
對於初學者，規律的生活和標準化的生活都是至關重要的。
而象形字是早期對於hu樹的簡化實踐。
著眼於標靶，自然而然地也許會失去對於自己的事情的認真。

Translations:
 Here are the translations in CSV format:

,**耗時**,Enduring Time
終止後，我們能夠踏實地享受舒適圈中的生活。,After it ends, we can truly enjoy life in the comfort zone.
踩出了舒適圈，我們便放棄了規律的行為。,Once we step out of the comfort zone, we abandon regular behavior.
我開始反思，什麼是規律的行為？是怎麼樣才能讓自己跟海一樣深。,I started to reflect, what is regular behavior? How can I become as deep as the sea.
到底什麼是舒適圈，我們沒有什麼理由去

In [32]:
flashcards_csv

(',**耗時**\n终止後，我們能夠踏實地享受舒適圈中的生活。\n踩出了舒適圈，我們便放棄了規律的行為。\n我開始反思，什麼是規律的行為？是怎麼樣才能讓自己跟海一樣深。\n到底什麼是舒適圈，我們沒有什麼理由去破壞它，因為它終究是我們的最安全的地方。\n規律就是標準化的習慣。\n規律就是我的標準化的生活。\n標準化的生活就是規律的生活。\n規律的生活中，我們能夠享受舒適圈中的自由。\n但是我們的生活中應該是不斷地學習和反思，不斷地挑戰舒適圈中的舒適。\n如果生活太舒適了，就會失去創造力的，只能走在平坦的道路上。\n而只有意識到規律才能實現舒適圈中的自由生活。\n我們應該走出去，去探索新的事物，於是我們才能夠將自己的喜好變成舒適圈中的要素。\n但是我們也不能太過於跟著課本大綱走，因為它會使我們缺乏思考的能力和創造力。\n但是在生活中，我們還是需要規律的生活，而標準化的生活也許就是我的希望。\n如果我的生活和我的規律之路太過於吻合，就有可能失去自己的意識。\n如果我們沒有規律的生活，就會走向失常的生活，失常的想法。\n但是，我們也不能再太過於依賴舒適圈中的自信，因為它不是實際的力量，實際的力量只有通過自己的事情實現。\n我們的舒適圈不應該是標靶，舒適圈只是一個可怕的陷阱。\n對於初學者，規律的生活和標準化的生活都是至關重要的。\n而象形字是早期對於hu樹的簡化實踐。\n著眼於標靶，自然而然地也許會失去對於自己的事情的認真。',
 "Here are the translations in CSV format:\n\n,**耗時**,Enduring Time\n終止後，我們能夠踏實地享受舒適圈中的生活。,After it ends, we can truly enjoy life in the comfort zone.\n踩出了舒適圈，我們便放棄了規律的行為。,Once we step out of the comfort zone, we abandon regular behavior.\n我開始反思，什麼是規律的行為？是怎麼樣才能讓自己跟海一樣深。,I started to reflect, what is regular behavior? How can I become as deep as the sea.\n到底什麼是舒適圈，我們沒有什麼理由去破壞它

In [57]:
import requests
import json
import os

# Load API key from environment variable
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

def organize_and_translate_notes_with_pinyin(notes, prompt):
    # Define the OpenRouter API endpoint
    url = "https://openrouter.ai/api/v1/chat/completions"

    # Define the data payload
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",  # Optional to specify the model
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "top_p": 1,
        "temperature": 0.8,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "repetition_penalty": 1,
        "top_k": 0,
    }

    # Send the request to the API
    response = requests.post(
        url=url,
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        data=json.dumps(data)
    )

    # Handle the response and return the content
    if response.status_code == 200:
        result = response.json()
        # Extract and return the AI-generated text (CSV output)
        return result['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [58]:
notes = """規律
踏實
舒適圈
自信
不知所措
裝潢
裝修
翻新
廣泛guang3fan4
籠統
跟海一樣深
終究
終止
初學者
象形字
hu樹
火 日 月 犬 馬 魚 舌 口 一
規則
敘事
跟著課本大綱走
標準化
脆弱、薄弱 
錯愕
有利
不是自己產生興趣
意識
""" 

In [67]:
prompt = (
    f"You are given a list of Mandarin vocabulary: {notes}. "
    "Your task is to clean and process the notes and extract each vocabulary word for mandarin revision, provide the English translation, and its pinyin. "
    "Format the output as a CSV where each row represents a flashcard. "
    "The first column should contain the Mandarin word, and the second column should contain the pinyin."
    "and the third column should contain its English meanings."
    "Please provide **only** the CSV data without any additional text or explanations. "
    "The CSV should look like this:\n"
    "Mandarin,Pinyin,English\n"
)


In [68]:
output = organize_and_translate_notes_with_pinyin(notes, prompt)
print(output)

規律,guī lǜ,regularity
踏實,tà shí,steadfast
舒適圈,shū shì quān,comfort zone
自信,zì xìn,self-confidence
不知所措,bù zhī suǒ cuò,at a loss
裝潢,zhuāng huàn,decorating
裝修,zhuāng xiū,renovation
翻新,fāng xīn,renovation
廣泛,guǎng fàn,widespread
籠統,lǒng tǒng,vague
跟海一樣深,gēn hǎi yī yàng shēn,as deep as the sea
終究,zhōng jiu,eventually
終止,zhōng zhǐ,to stop
初學者,chū xué zhě,beginner
象形字,xiàng xíng zì,pictorial characters
hu樹,hú shù,(no translation)
火,huǒ,fire
日,rì,sun/day
月,yuè,moon
犬,quǎn,dog
馬,mǎ,horse
魚,yú,fish
舌,shé,tongue
口,kǒu,mouth
一,yī,one
規則,guī zé,rule
敘事,xù shì,narrative
跟著課本大綱走,gēn zhe kè běn dà gāng zǒu,follow the textbook outline
標準化,bīng zhǔn huà,standardization
脆弱、薄弱,chuī ruò bǐ ruò,fragile
錯愕,cuò gào,astonished
有利,yǒu lì,beneficial
不是自己產生興趣,bù shì zì jǐ chǎn shēng xīng qù,not generated by one's own interest
意識,yì shí,consciousness


In [ ]:
prompt = (
    f"You are given a list of Mandarin vocabulary: {notes}. "
    "Your task is to extract the vocabulary and provide the English translation. "
    "Format the output as a CSV where each row represents a flashcard. "
    "The first column should contain the Mandarin word, and the second column should contain its English translation. "
    "Please provide **only** the CSV data without any additional text or explanations. "
    "The CSV should look like this:\n"
    "Mandarin,English\n"
)


In [8]:
import requests
import json
import os

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Step 1: Organize vocabulary into sentences
def organize_vocabulary(notes):https://www.promptingguide.ai/
    url = "https://openrouter.ai/api/v1/chat/completions"
    prompt_group_vocab = (
        f"You are given the following list of Mandarin notes which consist mostly of vocabulary and some occasional phrases: {notes}. "
        "Your task is to organize the words into coherent sentences. Please group together different vocabulary that can fit into the same sentence, aiming to minimize the total number of sentences. "
        "The sentences should be useful for an upper-intermediate Mandarin learner to remember the vocabulary."
    )
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_group_vocab}],
    }
    
    response = requests.post(
        url=url,
        headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"},
        data=json.dumps(data)
    )
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        print(f"Error in Step 1: {response.status_code}, {response.text}")
        return None

# Step 2: Translate sentences into English
def translate_sentences(sentences):
    url = "https://openrouter.ai/api/v1/chat/completions"
    prompt_translate = f"Translate the following Mandarin sentences into English:\n\n{sentences}. Provide the translations in a list format."
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_translate}],
    }
    
    response = requests.post(
        url=url,
        headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"},
        data=json.dumps(data)
    )
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        print(f"Error in Step 2: {response.status_code}, {response.text}")
        return None

# Step 3: Generate vocabulary dictionary with pinyin and translations
def generate_vocab_dictionary(sentences):
    url = "https://openrouter.ai/api/v1/chat/completions"
    prompt_vocab_dictionary = (
        f"From the following Mandarin sentences:\n\n{sentences}, "
        "produce a dictionary containing key vocabulary from the sentences, including the pinyin and English translation. "
        "Only include words that an upper-intermediate learner would find useful."
    )
    
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",
        "messages": [{"role": "user", "content": prompt_vocab_dictionary}],
    }
    
    response = requests.post(
        url=url,
        headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"},
        data=json.dumps(data)
    )
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        print(f"Error in Step 3: {response.status_code}, {response.text}")
        return None

# Example usage with Mandarin notes
notes = "苹果 桌子 学生 老师 我们 去 看电影"

# Step 1: Group vocabulary into sentences
grouped_sentences = organize_vocabulary(notes)
if grouped_sentences:
    print("Grouped Sentences:")
    print(grouped_sentences)

    # Step 2: Translate the sentences
    translations = translate_sentences(grouped_sentences)
    if translations:
        print("Translations:")
        print(translations)

        # Step 3: Generate a vocabulary dictionary
        vocab_dict = generate_vocab_dictionary(grouped_sentences)
        if vocab_dict:
            print("Vocabulary Dictionary:")
            print(vocab_dict)


Grouped Sentences:
Here are the sentences grouped together to minimize the total number of sentences and make them useful for an upper-intermediate Mandarin learner:

1. 我们去看电影。 (Wǒmen qù kàn diànyǐng.) - We are going to the movies.
2. 老师和学生在桌子上。 (Lǎoshī hé xuéshēng zài zhuōzi shàng.) - The teacher and students are at the desk.

 Sentence 1 uses the vocabulary:
- 我们 (wǒmen) - we
- 去 (qù) - to go
- 看 (kàn) - to watch/view
- 电影 (diànyǐng) - movie

 Sentence 2 uses the vocabulary:
- 老师 (lǎoshī) - teacher
- 学生 (xuéshēng) - student
- 桌子 (zhuōzi) - desk 

The vocabulary 苹果 (píngguǒ) - apple is not used in the sentences above, so I would group it with a new sentence that combines it with the other vocabulary not used in the previous sentences. 

3. 我爱吃苹果。 (Wǒ ài chī píngguǒ.) - I love eating apples.

 Sentence 3 uses the vocabulary:
- 我爱 (wǒ ài) - I love
- 吃 (chī) - to eat
- 苹果 (píngguǒ) - apple
Translations:
Here are the translations of the three Mandarin sentences:

**Sentence 1:** We are g

In [4]:
import requests
import json
import os

# Load API key from environment variable
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

def organize_and_translate_notes_with_pinyin(notes):
    # Define the OpenRouter API endpoint
    url = "https://openrouter.ai/api/v1/chat/completions"

    # The content prompt: group Mandarin words, translate, and return CSV
    prompt = (
        f"You are given the following list of Mandarin words: {notes}. "
        "Your task is to intelligently group related words into meaningful sentences to minimize the number of flashcards. "
        "Please create a CSV with three columns:\n"
        "1. The Mandarin sentence\n"
        "2. The English translation of the sentence\n"
        "3. A list of key Mandarin words from the sentence, with their pinyin and English translation.\n"
        "If a word does not fit well into a sentence, create a single flashcard for it."
    )

    # Define the data payload
    data = {
        "model": "meta-llama/llama-3.1-8b-instruct:free",  # Optional to specify the model
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "top_p": 1,
        "temperature": 0.8,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "repetition_penalty": 1,
        "top_k": 0,
    }

    # Send the request to the API
    response = requests.post(
        url=url,
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        data=json.dumps(data)
    )

    # Handle the response and return the content
    if response.status_code == 200:
        result = response.json()
        # Extract and return the AI-generated text (CSV output)
        return result['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [6]:
# Example usage with Mandarin notes
notes = """規律
踏實
舒適圈
自信
不知所措
裝潢
裝修
翻新
廣泛guang3fan4
籠統
跟海一樣深
終究
終止
初學者
象形字
hu樹
火 日 月 犬 馬 魚 舌 口 一
規則
敘事
跟著課本大綱走
標準化
脆弱、薄弱 
錯愕
有利
不是自己產生興趣
意識
"""  # Example words in Mandarin

In [7]:
output = organize_and_translate_notes_with_pinyin(notes)

if output:
    # Print the CSV format output
    print("Generated CSV:")
    print(output)

Generated CSV:
After analyzing the given list of Mandarin words, I've created the following CSV file with grouped sentences and key words:

| Mandarin Sentence | English Translation | Key Mandarin Words (Pinyin and English Translation) |
| --- | --- | --- |
| 規律很重要 | Rules are important | 規律 (guī lǜ, rule) |
|  |  |  |
| 必须跟著課本大綱走才能掌握規律 | You must follow the course outline to grasp the rules | 跟著 (gēn zhe, following),課本 (kè běn, textbook),大綱 (dà gāng, outline) |
| 規律是很重要的基礎 | Rules are a fundamental basis | 規律 (guī lǜ, rule) |
|  |  |  |
| 如果不學習規則，就很難掌握規律 | If you don't learn the rules, it's hard to grasp them | 規則 (guī zé, rule), 規律 (guī lǜ, rule) |
|  |  |  |
| 很多人在學習規律時感到不知所措 | Many people feel lost when learning rules | 知不知所措 (zhī bù zhī suǒ cuò, feeling lost) |
|  |  |  |
| 很多規律都是標準化的 | Many rules are standardized | 標準化 (biāo zhǔn huà, standardization) |
|  |  |  |
| 標準化的規律很有利 | Standardized rules are beneficial | 標準化 (biāo zhǔn huà, standardization), 有利 (yǒu lì, beneficial) |
|  

In [4]:
import requests
import json
import os

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": "meta-llama/llama-3.1-8b-instruct:free", # Optional
    "messages": [
      {"role": "user", "content": "What is the meaning of life?"}
    ],
    "top_p": 1,
    "temperature": 0.8,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "repetition_penalty": 1,
    "top_k": 0,
  })
)

In [6]:
response.content

b'\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n         \n\n        